In [1]:
import optunity
import optunity.cross_validation
import optunity.metrics
import numpy as np
import sklearn.svm

import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import numpy as np
import operator

class MajorityVoteClassifier(BaseEstimator,
                             ClassifierMixin):
    """ A majority vote ensemble classifier
    Parameters
    ----------
    classifiers : array-like, shape = [n_classifiers]
      Different classifiers for the ensemble
    vote : str, {'classlabel', 'probability'} (default='label')
      If 'classlabel' the prediction is based on the argmax of
        class labels. Else if 'probability', the argmax of
        the sum of probabilities is used to predict the class label
        (recommended for calibrated classifiers).
    weights : array-like, shape = [n_classifiers], optional (default=None)
      If a list of `int` or `float` values are provided, the classifiers
      are weighted by importance; Uses uniform weights if `weights=None`.
    """
    def __init__(self, classifiers, vote='classlabel', weights=None):

        self.classifiers = classifiers
        self.named_classifiers = {key: value for key, value
                                  in _name_estimators(classifiers)}
        self.vote = vote
        self.weights = weights

    def fit(self, X, y):
        """ Fit classifiers.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Matrix of training samples.
        y : array-like, shape = [n_samples]
            Vector of target class labels.
        Returns
        -------
        self : object
        """
        if self.vote not in ('probability', 'classlabel'):
            raise ValueError("vote must be 'probability' or 'classlabel'"
                             "; got (vote=%r)"
                             % self.vote)

        if self.weights and len(self.weights) != len(self.classifiers):
            raise ValueError('Number of classifiers and weights must be equal'
                             '; got %d weights, %d classifiers'
                             % (len(self.weights), len(self.classifiers)))

        # Use LabelEncoder to ensure class labels start with 0, which
        # is important for np.argmax call in self.predict
        self.lablenc_ = LabelEncoder()
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
        for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X, self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self

    def predict(self, X):
        """ Predict class labels for X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Matrix of training samples.
        Returns
        ----------
        maj_vote : array-like, shape = [n_samples]
            Predicted class labels.
        """
        if self.vote == 'probability':
            maj_vote = np.argmax(self.predict_proba(X), axis=1)
        else:  # 'classlabel' vote

            #  Collect results from clf.predict calls
            predictions = np.asarray([clf.predict(X)
                                      for clf in self.classifiers_]).T

            maj_vote = np.apply_along_axis(
                lambda x:
                np.argmax(np.bincount(x,
                                      weights=self.weights)),
                axis=1,
                arr=predictions)
        maj_vote = self.lablenc_.inverse_transform(maj_vote)
        return maj_vote

    def predict_proba(self, X):
        """ Predict class probabilities for X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        avg_proba : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.
        """
        probas = np.asarray([clf.predict_proba(X)
                             for clf in self.classifiers_])
        avg_proba = np.average(probas, axis=0, weights=self.weights)
        return avg_proba

    def get_params(self, deep=True):
        """ Get classifier parameter names for GridSearch"""
        if not deep:
            return super(MajorityVoteClassifier, self).get_params(deep=False)
        else:
            out = self.named_classifiers.copy()
            for name, step in six.iteritems(self.named_classifiers):
                for key, value in six.iteritems(step.get_params(deep=True)):
                    out['%s__%s' % (name, key)] = value
        return out
    
def creatematrix(features, kmer):
    stat = pd.read_csv(features, sep=",")
    kmer = pd.read_csv(kmer, sep="\t", header = None)
    kmer = kmer.iloc[:, :-1]
    id2 = stat.id.str.split("-",expand=True) # split the string to get label
    id2 = id2.iloc[:, :-1]
    stat2 = stat.iloc[:, 1:]
    df = pd.concat([stat2.reset_index(drop=True), kmer], axis=1) # concat kmerand stat matrix
    df = pd.concat([id2, df], axis=1)
    df.columns.values[0] = "label"
    # encoding class labels as integers
    df.loc[df.label == 'positive', 'label'] = 1
    df.loc[df.label == 'negative', 'label'] = 0
    return df

print("load data")
dat = creatematrix('../dat/train.features.clear2.csv', '../dat/train.features.kmer')

# check if the dataset is inbalanced
dat.pos = dat.loc[dat['label'] == 1]
dat.neg = dat.loc[dat['label'] == 0]

#dat.pos.shape
print('number of negative instances: %d ' % (dat.neg.shape[0]))
print('number of positive instances: %d ' % (dat.pos.shape[0]))
#num = min(dat.neg.shape[0],dat.pos.shape[0])
#print('limit dataset size to %d' % (num))

# generate a random subset of both with the size of $num

#posrand = dat.pos.sample(n=num)
#negrand = dat.neg.sample(n=num)

#df = posrand.copy()
#dat = df.append(negrand)

# split to test and training set
y = dat['label'].tolist() # extract label
X = dat.drop(dat.columns[[0]], 1) # remove label

#data = X.as_matrix()
#labels = [True] * len(posrand) + [False] * len(negrand)
print('done')





/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


load data
number of negative instances: 94168 
number of positive instances: 2117 
done


In [2]:
def balanced_subsample(x,y,subsample_size=1.0):

    class_xs = []
    min_elems = None

    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]

    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)

    xs = []
    ys = []

    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
            this_xs = this_xs.reindex(np.random.permutation(this_xs.index))
            
        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    xs = pd.concat(xs)
    ys = pd.Series(data=np.concatenate(ys),name='target')

    return xs,ys

In [3]:
X_sub, y_sub = balanced_subsample(X, y, subsample_size=0.1)

In [4]:
import numpy as np

from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [5, 4, 3, None],
              "n_estimators": [500, 2000],
              "max_features": sp_randint(1, 50),
              "min_samples_split": sp_randint(2, 50),
              "min_samples_leaf": sp_randint(1, 50),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 10
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, scoring = 'accuracy',
                                   n_iter=n_iter_search, n_jobs=-1, refit=True)

start = time()
random_search.fit(X_sub, y_sub)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)


RandomizedSearchCV took 48.56 seconds for 10 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.711 (std: 0.033)
Parameters: {'bootstrap': False, 'min_samples_leaf': 8, 'n_estimators': 500, 'min_samples_split': 36, 'criterion': 'gini', 'max_features': 19, 'max_depth': 3}

Model with rank: 2
Mean validation score: 0.694 (std: 0.009)
Parameters: {'bootstrap': False, 'min_samples_leaf': 13, 'n_estimators': 500, 'min_samples_split': 43, 'criterion': 'gini', 'max_features': 3, 'max_depth': None}

Model with rank: 3
Mean validation score: 0.692 (std: 0.031)
Parameters: {'bootstrap': False, 'min_samples_leaf': 38, 'n_estimators': 500, 'min_samples_split': 46, 'criterion': 'entropy', 'max_features': 17, 'max_depth': 4}



In [23]:
np.logspace(-9, 3, 13)
np.logspace(-2, 10, 13)

array([  1.00000000e-09,   1.00000000e-08,   1.00000000e-07,
         1.00000000e-06,   1.00000000e-05,   1.00000000e-04,
         1.00000000e-03,   1.00000000e-02,   1.00000000e-01,
         1.00000000e+00,   1.00000000e+01,   1.00000000e+02,
         1.00000000e+03])

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = LogisticRegression(penalty='l2')

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# specify parameters and distributions to sample from
param_dist = {"C": np.logspace(-9, 3, 13),
              "solver": ['newton-cg', 'lbfgs', 'liblinear', 'sag'],
              "dual":[False],
              "tol": np.logspace(-9, 3, 13)
            }

# run randomized search
n_iter_search = 200
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, scoring = 'accuracy',
                                   n_iter=n_iter_search, n_jobs=-1, refit=True)

start = time()
random_search.fit(X_sub, y_sub)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

/usr/local/lib/python2.7/dist-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/usr/local/lib/python2.7/dist-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/usr/local/lib/python2.7/dist-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search fail

RandomizedSearchCV took 42.41 seconds for 200 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.704 (std: 0.041)
Parameters: {'C': 0.01, 'dual': False, 'tol': 1.0000000000000001e-05, 'solver': 'sag'}

Model with rank: 2
Mean validation score: 0.701 (std: 0.044)
Parameters: {'C': 0.001, 'dual': False, 'tol': 1.0, 'solver': 'liblinear'}

Model with rank: 3
Mean validation score: 0.699 (std: 0.043)
Parameters: {'C': 0.01, 'dual': False, 'tol': 9.9999999999999995e-07, 'solver': 'sag'}

Model with rank: 3
Mean validation score: 0.699 (std: 0.046)
Parameters: {'C': 0.01, 'dual': False, 'tol': 1e-08, 'solver': 'sag'}



In [32]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

scores = cross_val_score(estimator=clf,
                             X=X_sub,
                             y=y_sub,
                             cv=10,
                             scoring='accuracy')
print("ROC AUC: %0.2f (+/- %0.2f)"
          % (scores.mean(), scores.std()))


ROC AUC: 0.65 (+/- 0.06)


In [46]:
from sklearn.svm import SVC

clf = SVC()

# specify parameters and distributions to sample from
param_dist = {'C': pow(2.0, np.arange(-10, 11, 0.1)), 'gamma': pow(2.0, np.arange(-10, 11, 0.1)),
'kernel': ['linear', 'rbf']}

# run randomized search
n_iter_search = 30
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, scoring = 'accuracy',
                                   n_iter=n_iter_search, n_jobs=-1, refit=True)

start = time()
random_search.fit(X_sub, y_sub)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

RandomizedSearchCV took 18.17 seconds for 30 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.692 (std: 0.030)
Parameters: {'kernel': 'linear', 'C': 0.0014801919594828084, 'gamma': 362.03867196749627}

Model with rank: 2
Mean validation score: 0.671 (std: 0.025)
Parameters: {'kernel': 'rbf', 'C': 6.0628662660414046, 'gamma': 0.0029603839189656098}

Model with rank: 2
Mean validation score: 0.671 (std: 0.035)
Parameters: {'kernel': 'linear', 'C': 0.0029603839189656098, 'gamma': 891.44377681518768}

